# Importing Modules Example
## Introduction
In certain situations it will be convenient to separate the functionality of a task into multiple python modules or files. In this example, we create a function which imports another module to execute its functionality. Specifically, the function squares a number by importing a tranform helper function from another python module in the same folder as the [handler.py](handler.py) main module. 

**Documentation**:
We will be using the Python SDK of Cognite Functions for which the documentation can be found here:

https://cognite-sdk-experimental.readthedocs-hosted.com/en/latest/cognite.html#functions

## Setup
We first import the experimental CogniteClient and need to pass the client an api-key, which we retrieve via `getpass`.

In [ ]:
from getpass import getpass
from cognite.experimental import CogniteClient

api_key = getpass()
client = CogniteClient(
    api_key=api_key,
    project="functions-tutorial",
    client_name="DSHub",
    base_url="https://greenfield.cognitedata.com"
)

## Defining our function

In this example we define a function in two python modules, a [handler.py](handler.py) which contains the main `handle` function, which always needs to be present when defining a new function, and a [helper.py](helper.py) module. The latter module is called by the former to show how module dependencies can be used. The helper module is imported in the handler by writing `import helper` which we can do because it's contained in the same folder as the [handler.py](handler.py) module.

Note that in [handler.py](handler.py), the function `handle` must have as arguments a subset of `(client, data, secrets)`. In this example we only need `data`. The data we give when calling the function, is passed through the `data` argument. 

## Testing the function locally
Before we actually deploy the function to CDF, we can test it locally. First take a look at the two files [handler.py](handler.py) and [helper.py](helper.py), before you test the code.

In [ ]:
import handler
handler.handle(data={"value": 2.0})

## Deploying the function to CDF
Next we deploy our function to CDF, by setting the argument `folder` to the relevant folder. At least one of the files in the folder must be named `handler.py`, and this file must have a function named `handle`, which is the function that will be executed in CDF.

First let's create a unique string as a function external id (e.g. your own name if it's unique to cognite)

In [ ]:
my_name =  # put your name as string here
external_id = f"my-transformer-{my_name}"

In [ ]:
function = client.functions.create(
    name=external_id, # We just use external id as name so it is easier to find in Data Studio UI
    external_id=external_id,
    folder='.', # We will upload this directory
    description="Square an input value", 
    owner="kir@aker.com")

Next we can retrieve the function to see its status. It will start out as `Queued` and then go to `Deploying`. After a couple of minutes, the status will be `Ready`, and you can start to call your function. **NB:** All calls to the function before the function is `Ready` will fail.

In [ ]:
# Repeat until status is ready
function = client.functions.retrieve(external_id=external_id)
function

When the function is `Ready`, we can call it directly on the function object.

In [ ]:
call1 = function.call(data={"value": 2})
call1

Note that the above calls are to the function in CDF, not the local function.

To retrieve the response from the function, simply invoke the `get_response()` method on the call object:

In [ ]:
response = call1.get_response()
response

## Handling failures
If you get a failure you can look at the logs

In [ ]:
call_bad = function.call(data={"foo": 2})
call_bad

In [ ]:
call_bad.get_logs()

In [ ]:
call_bad.get_logs()[-2:] # Here we see the error

### Schedules

Schedules are objects that make the functions run at pre-defined intervals. For example, we can run a function every Monday at 7am or every other friday, etc. A schedule is defined with an associated function's external id and some input data. The interval syntax we use is called unix cron syntax, which is a string that contains the necessary information to decode a wide variaty of running intervals. More information about cron syntax can be found here: https://crontab.guru.

Below is an example of a schedule that runs every minute on our defined function. Note that we can define multiple schedules on each function, each one with different input data.

In [ ]:
schedule = client.functions.schedules.create(
    name=f"Schedule 1", 
    cron_expression="* * * * *", 
    function_external_id=external_id,
    data={"value": 2})
schedule

## List calls
We can list all calls that have been made to the function:

In [ ]:
calls = function.list_calls()
calls

In [ ]:
calls[-1].get_response()

## Cleanup
Finally, we clean after ourselves by deleting our time series and function.

In [ ]:
client.functions.schedules.delete(id=schedule.id)

In [ ]:
client.functions.delete(external_id=external_id)

In [ ]:
client.functions.list()